In [1]:
# Importing the required libraries 
import nltk
import numpy as np
import itertools

In [2]:
# Downloading and importing the brown corpus
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
#Getting the tagged sentences
sent_tag = brown.tagged_sents()
mod_sent_tag=[]
for s in sent_tag:
  s.insert(0,('##','##'))
  s.append(('&&','&&'))
  mod_sent_tag.append(s)

In [4]:
#Splitting the data for train and test
split_num = int(len(mod_sent_tag)*0.9)
train_data = mod_sent_tag[0:split_num]
test_data = mod_sent_tag[split_num:]

In [19]:
#Creating a dictionary whose keys are tags and values contain words which were assigned the correspoding tag
# ex:- 'TAG':{word1: count(word1,'TAG')}
train_word_tag = {}
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      try:
        train_word_tag[t][w]+=1
      except:
        train_word_tag[t][w]=1
    except:
      train_word_tag[t]={w:1}
train_word_tag

{'##': {'##': 51606},
 'AT': {'the': 64805,
  'an': 3456,
  'no': 1649,
  'a': 20956,
  'every': 446,
  "th'": 1},
 'NP-TL': {'fulton': 10,
  'atlanta': 4,
  'grady': 4,
  'georgia': 14,
  'jackson': 2,
  'miller': 2,
  'colquitt': 1,
  'texas': 17,
  'dallas': 13,
  'beaumont': 1,
  'lamar': 2,
  'texan': 1,
  'wise': 1,
  'paris': 3,
  'oklahoma': 3,
  'rhode': 99,
  'massachusetts': 14,
  'u.s.': 56,
  'denton': 1,
  'york': 285,
  'st.': 11,
  'louis': 13,
  'cook': 2,
  'republican': 9,
  'atlantic': 6,
  'viet': 10,
  'nam': 9,
  'lao': 3,
  'boston': 11,
  'america': 73,
  'asia': 18,
  'providence': 38,
  'narragansett': 4,
  'sheraton-biltmore': 3,
  'johnston': 4,
  'westfield': 2,
  'jersey': 19,
  'mack': 1,
  'kennedy': 10,
  'essex': 5,
  'orange': 5,
  'michigan': 5,
  'morris': 3,
  'brooklyn': 16,
  'feis': 1,
  'hunter': 3,
  'u.': 36,
  's.': 37,
  'times-picayune': 1,
  'orleans': 39,
  'miles': 1,
  'pennsylvania': 14,
  'columbia': 16,
  'lafayette': 6,
  'samoa':

In [20]:
#Calculating the emission probabilities using train_word_tag
train_emission_prob={}
for k in train_word_tag.keys():
  train_emission_prob[k]={}
  count = sum(train_word_tag[k].values())
  for k2 in train_word_tag[k].keys():
    train_emission_prob[k][k2]=train_word_tag[k][k2]/count
train_emission_prob

{'##': {'##': 1.0},
 'AT': {'the': 0.7097017949251476,
  'an': 0.037847842037825936,
  'no': 0.018058764907515908,
  'a': 0.22949634772704872,
  'every': 0.004884299059279621,
  "th'": 1.0951343182241303e-05},
 'NP-TL': {'fulton': 0.0025886616619207868,
  'atlanta': 0.0010354646647683149,
  'grady': 0.0010354646647683149,
  'georgia': 0.0036241263266891016,
  'jackson': 0.0005177323323841574,
  'miller': 0.0005177323323841574,
  'colquitt': 0.0002588661661920787,
  'texas': 0.004400724825265338,
  'dallas': 0.003365260160497023,
  'beaumont': 0.0002588661661920787,
  'lamar': 0.0005177323323841574,
  'texan': 0.0002588661661920787,
  'wise': 0.0002588661661920787,
  'paris': 0.000776598498576236,
  'oklahoma': 0.000776598498576236,
  'rhode': 0.02562775045301579,
  'massachusetts': 0.0036241263266891016,
  'u.s.': 0.014496505306756407,
  'denton': 0.0002588661661920787,
  'york': 0.07377685736474243,
  'st.': 0.0028475278281128655,
  'louis': 0.003365260160497023,
  'cook': 0.000517732

In [7]:
#Estimating the bigram of tags to be used for transition probability
bigram_tag_data = {}
for s in train_data:
  bi=list(nltk.bigrams(s))
  for b1,b2 in bi:
    try:
      try:
        bigram_tag_data[b1[1]][b2[1]]+=1
      except:
        bigram_tag_data[b1[1]][b2[1]]=1
    except:
      bigram_tag_data[b1[1]]={b2[1]:1}

In [8]:
#Calculating the probabilities of tag bigrams for transition probability  
bigram_tag_prob={}
for k in bigram_tag_data.keys():
  bigram_tag_prob[k]={}
  count=sum(bigram_tag_data[k].values())
  for k2 in bigram_tag_data[k].keys():
    bigram_tag_prob[k][k2]=bigram_tag_data[k][k2]/count

In [9]:
#Calculating the possible tags for each word
#Note: Here we have used the whole data(Train+Test)
#Reason: There may be some words which are not present in train data but are present in test data 
tags_of_tokens = {}
count=0
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l
        
for s in test_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l

In [10]:
#Dividing the test data into test words and test tags
test_words=[]
test_tags=[]
for s in test_data:
  temp_word=[]
  temp_tag=[]
  for (w,t) in s:
    temp_word.append(w.lower())
    temp_tag.append(t)
  test_words.append(temp_word)
  test_tags.append(temp_tag)

In [17]:
#Executing the Viterbi Algorithm
predicted_tags = []                #intializing the predicted tags
for x in range(len(test_words)):   # for each tokenized sentence in the test data
  s = test_words[x]
  #storing_values is a dictionary which stores the required values
  #ex: storing_values = {step_no.:{state1:[previous_best_state,value_of_the_state]}}                
  storing_values = {}              
  for q in range(len(s)):
    step = s[q]
    #for the starting word of the sentence
    if q == 1:                
      storing_values[q] = {}
      tags = tags_of_tokens[step]
      for t in tags:
        #this is applied since we do not know whether the word in the test data is present in train data or not
        try:
          storing_values[q][t] = ['##',bigram_tag_prob['##'][t]*train_emission_prob[t][step]]
        #if word is not present in the train data but present in test data we assign a very low probability of 0.0001
        except:
          storing_values[q][t] = ['##',0.0001]#*train_emission_prob[t][step]]
    
    #if the word is not at the start of the sentence
    if q>1:
      storing_values[q] = {}
      previous_states = list(storing_values[q-1].keys())   # loading the previous states
      current_states  = tags_of_tokens[step]               # loading the current states
      #calculation of the best previous state for each current state and then storing
      #it in storing_values
      for t in current_states:                             
        temp = []
        for pt in previous_states:                         
          try:
            temp.append(storing_values[q-1][pt][1]*bigram_tag_prob[pt][t]*train_emission_prob[t][step])
          except:
            temp.append(storing_values[q-1][pt][1]*0.0001)
        
        max_temp_index = temp.index(max(temp))
        best_pt = previous_states[max_temp_index]
        storing_values[q][t]=[best_pt,max(temp)]
    #print(storing_values)
    #print('---------------')
  #Backtracing to extract the best possible tags for the sentence
  pred_tags = []
  total_steps_num = storing_values.keys()
  #print(total_steps_num)
  last_step_num = max(total_steps_num)
  for bs in range(len(total_steps_num)):
    step_num = last_step_num - bs
    if step_num == last_step_num:
      pred_tags.append('&&')
      pred_tags.append(storing_values[step_num]['&&'][0])
    if step_num<last_step_num and step_num>0:
      pred_tags.append(storing_values[step_num][pred_tags[len(pred_tags)-1]][0])
  predicted_tags.append(list(reversed(pred_tags)))

print(predicted_tags)

[['##', 'PPS', 'BEDZ', 'RB', 'CD', 'NNS', 'JJ', '.', '&&'], ['##', '``', 'DT', 'JJ', 'NN', ',', 'PP$', 'UH', '.', '.', '&&'], ['##', 'RB', ',', 'PPSS', 'BER', 'QL', 'JJ', 'TO', 'VB', 'PPO', 'RB', 'PPL', "''", ',', 'PPS', 'VBD', ',', 'VBG', 'IN', 'NN', '.', '&&'], ['##', 'NP', 'VBD', 'PP$', 'NN', 'IN', 'AT', 'NN', ',', 'CC', 'AT', 'NN', 'VBD', 'PPO', 'CS', 'AT', 'JJ', 'NN', '.', '&&'], ['##', '``', 'NP', 'BEZ', 'TO', 'BE', 'PP$-HL', 'NN-HL', 'NP', ',', 'NP', '.', '&&'], ['##', 'PPSS', 'VB', 'AT', 'NN', 'IN', 'AT', 'NN', 'VBN', 'IN', 'PPO', 'PPS', 'BEZ', 'AT', 'QL', 'JJ', 'NN', ',', 'QL', 'JJ', 'IN', 'AP', 'NNS', '.', '&&'], ['##', 'PPS', 'VBZ', 'AT', 'JJR', 'NN', 'CS', 'RB', 'VBG', 'RB', 'IN', 'AT', 'NP', 'NN', "''", '.', '&&'], ['##', '``', 'QL', 'RB', ',', 'PP$', 'UH', '.', '&&'], ['##', 'PP$', 'NN', 'MD', 'BE', 'VB', 'RB', "''", '.', '&&'], ['##', 'AT', 'NN', 'VBD', 'AT', 'NNS', 'IN', 'AT', 'NN', 'IN', 'AT', 'JJ', 'NN', 'IN', 'AT', 'VBG', 'NN', '.', '&&'], ['##', 'HVD-HL', 'NP', 'BEN

In [18]:
#Calculating the accuracy based on tagging each word in the test data.
right = 0 
wrong = 0

for i in range(len(test_tags)):
  gt = test_tags[i]
  pred = predicted_tags[i]
  for h in range(len(gt)):
    if gt[h] == pred[h]:
      right = right+1
    else:
      wrong = wrong +1 

print('Accuracy on the test data is: ',right/(right+wrong))
print('Loss on the test data is: ',wrong/(right+wrong))

[['##', 'PPS', 'BEDZ', 'RB', 'CD', 'NNS', 'JJ', '.', '&&'], ['##', '``', 'DT', 'JJ', 'NN', ',', 'PP$', 'NN', '.', '.', '&&'], ['##', 'RB', ',', 'PPSS', 'BER', 'QL', 'JJ', 'TO', 'VB', 'PPO', 'RB', 'PPL', "''", ',', 'PPS', 'VBD', ',', 'VBG', 'IN', 'NN', '.', '&&'], ['##', 'NP', 'VBD', 'PP$', 'NN', 'IN', 'AT', 'NN', ',', 'CC', 'AT', 'NN', 'VBD', 'PPO', 'CS', 'AT', 'JJ', 'NN', '.', '&&'], ['##', '``', 'NP', 'BEZ', 'TO', 'BE', 'PP$', 'NN', 'NN', ',', 'NP', '.', '&&'], ['##', 'PPSS', 'VB', 'AT', 'NN', 'IN', 'AT', 'NN', 'VBN', 'IN', 'PPO', 'PPS', 'BEZ', 'AT', 'QL', 'JJ', 'NN', ',', 'QL', 'JJ', 'IN', 'AP', 'NNS', '.', '&&'], ['##', 'PPS', 'VBZ', 'AT', 'JJR', 'NN', 'CS', 'RB', 'VBG', 'RB', 'IN', 'AT', 'NP', 'NN', "''", '.', '&&'], ['##', '``', 'QL', 'RB', ',', 'PP$', 'NN', '.', '&&'], ['##', 'PP$', 'NN', 'MD', 'BE', 'JJ', 'RB', "''", '.', '&&'], ['##', 'AT', 'NN', 'VBD', 'AT', 'NNS', 'IN', 'AT', 'NN', 'CS', 'AT', 'JJ', 'NN', 'IN', 'AT', 'VBG', 'NN', '.', '&&'], ['##', 'HVD', 'NP', 'BEN', 'JJR',